In [5]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# Database connection parameters
user = 'root'
password = 'root'
host = 'localhost'
port = '1234'
database = 'ny_taxi'

# Create connection string
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Create engine
engine = create_engine(connection_string)



In [6]:
# Load taxi zone lookup data
print("Loading taxi zone lookup data...")
zones_df = pd.read_csv('trip.csv')
zones_df.to_sql('taxi_zones', engine, if_exists='replace', index=False)
print(f"Loaded {len(zones_df)} zone records")

Loading taxi zone lookup data...
Loaded 265 zone records


In [8]:
# Load green taxi data in chunks (file is large)
print("Loading green taxi data...")
chunk_size = 100000
chunks_processed = 0

for chunk in pd.read_csv('green_trip.csv', chunksize=chunk_size):
    # Convert datetime columns
    chunk['lpep_pickup_datetime'] = pd.to_datetime(chunk['lpep_pickup_datetime'])
    chunk['lpep_dropoff_datetime'] = pd.to_datetime(chunk['lpep_dropoff_datetime'])
    
    # Load chunk to database
    chunk.to_sql('green_taxi_trips', engine, if_exists='append', index=False)
    chunks_processed += 1
    print(f"Processed chunk {chunks_processed} ({len(chunk)} records)")

print("Data loading completed!")


Loading green taxi data...
Processed chunk 1 (100000 records)
Processed chunk 2 (100000 records)
Processed chunk 3 (100000 records)


C:\Users\HP\AppData\Local\Temp\ipykernel_9044\740327491.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('green_trip.csv', chunksize=chunk_size):


Processed chunk 4 (100000 records)
Processed chunk 5 (49063 records)
Data loading completed!


In [11]:
from sqlalchemy import text

# Verify data loaded correctly
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM green_taxi_trips"))
    taxi_count = result.scalar()
    print(f"Total taxi trips loaded: {taxi_count}")
    
    result = conn.execute(text("SELECT COUNT(*) FROM taxi_zones"))
    zone_count = result.scalar()
    print(f"Total zones loaded: {zone_count}")



Total taxi trips loaded: 449063
Total zones loaded: 265


In [13]:
print(" Question 3: Count records")
print("How many taxi trips were totally made on September 18th 2019?")

query3 = """
SELECT COUNT(*) as trip_count
FROM green_taxi_trips
WHERE DATE(lpep_pickup_datetime) = '2019-09-18'
  AND DATE(lpep_dropoff_datetime) = '2019-09-18'
"""

result3 = pd.read_sql(query3, engine)
print(f"Answer: {result3['trip_count'].iloc[0]}")


 Question 3: Count records
How many taxi trips were totally made on September 18th 2019?
Answer: 15612


In [14]:
print("\n Question 4: Longest trip for each day")
print("Which was the pick up day with the longest trip distance?")

query4 = """
SELECT DATE(lpep_pickup_datetime) as pickup_date,
       MAX(trip_distance) as max_distance
FROM green_taxi_trips
GROUP BY DATE(lpep_pickup_datetime)
ORDER BY max_distance DESC
LIMIT 1
"""

result4 = pd.read_sql(query4, engine)
print(f"Answer: {result4['pickup_date'].iloc[0]}")



 Question 4: Longest trip for each day
Which was the pick up day with the longest trip distance?
Answer: 2019-09-26


In [27]:
print("\n  Question 5: Three biggest pick up Boroughs")
print("Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?")

query5 = """
SELECT tz."Borough",
       SUM(gt."total_amount") as total_amount_sum
FROM green_taxi_trips gt
JOIN taxi_zones tz ON gt."PULocationID" = tz."LocationID"
WHERE DATE(gt."lpep_pickup_datetime") = '2019-09-18'
  AND tz."Borough" != 'Unknown'
GROUP BY tz."Borough"
HAVING SUM(gt."total_amount") > 50000
ORDER BY total_amount_sum DESC
"""


result5 = pd.read_sql(query5, engine)
print("Boroughs with total_amount > 50000:")
for idx, row in result5.iterrows():
    print(f"  {row['Borough']}: ${row['total_amount_sum']:,.2f}")




  Question 5: Three biggest pick up Boroughs
Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?
Boroughs with total_amount > 50000:
  Brooklyn: $96,333.24
  Manhattan: $92,271.30
  Queens: $78,671.71


In [19]:
from sqlalchemy import create_engine, text

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM green_taxi_trips LIMIT 1"))
    print(result.keys())


RMKeyView(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'])


In [22]:
from sqlalchemy import create_engine, text

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM taxi_zones LIMIT 1"))
    print(result.keys())

RMKeyView(['LocationID', 'Borough', 'Zone', 'service_zone'])


In [30]:
print("\n Question 6: Largest tip")
print("For passengers picked up in September 2019 in Astoria, which drop off zone had the largest tip?")

query6 = """
SELECT tz_dropoff."Zone" as dropoff_zone,
       MAX(gt."tip_amount") as max_tip
FROM green_taxi_trips gt
JOIN taxi_zones tz_pickup ON gt."PULocationID" = tz_pickup."LocationID"
JOIN taxi_zones tz_dropoff ON gt."DOLocationID" = tz_dropoff."LocationID"
WHERE tz_pickup."Zone" = 'Astoria'
  AND DATE(gt."lpep_pickup_datetime") BETWEEN '2019-09-01' AND '2019-09-30'
GROUP BY tz_dropoff."Zone"
ORDER BY max_tip DESC
LIMIT 1
"""

result6 = pd.read_sql(query6, engine)
print(f"Answer: {result6['dropoff_zone'].iloc[0]}")
print(f"Tip amount: ${result6['max_tip'].iloc[0]:.2f}")




 Question 6: Largest tip
For passengers picked up in September 2019 in Astoria, which drop off zone had the largest tip?
Answer: JFK Airport
Tip amount: $62.31


In [31]:
print("\n Summary of Answers")
print(f"Question 3: {result3['trip_count'].iloc[0]}")
print(f"Question 4: {result4['pickup_date'].iloc[0]}")
print(f"Question 5: {', '.join(['\"' + str(row['Borough']) + '\"' for _, row in result5.iterrows()])}")
print(f"Question 6: {result6['dropoff_zone'].iloc[0]}")



 Summary of Answers
Question 3: 15612
Question 4: 2019-09-26
Question 5: "Brooklyn", "Manhattan", "Queens"
Question 6: JFK Airport
